# Algoritmia
## Práctica 4
El objetivo de esta práctica es implementar algunos métodos voraces.

En el cuerpo de cada función a implementar hay una instrucción "pass", se debe sustituir por la implementación adecuada. 

Para cada clase o función que se pide se proporcionan algunos tests. Las implementaciones deberían superar estos tests.

## Preámbulo

In [1]:
# Importaciones
import unittest
from collections import Counter

## Reparto de elementos

Se dispone de varios elementos de valor numérico y de contenedores de una determinada capacidad (todos la misma).

Los contenedores pueden almacenar elementos, siempre que la suma de elementos no supere su capacidad.

Por ejemplo, los elementos podrían ser ficheros, los valores numéricos sus tamaños y los contenedores serían discos.

Otro ejemplo serían objetos físicos, los valores serían sus pesos y los contenedores serían mochilas con una determinado peso máximo.

Se quiere repartir los elementos en contenedores, de manera que todos los elementos estén en alguno de los contenedores.

El objetivo sería minimizar el número de contenedores.

Se plantea una solución voraz:
- Mientras haya elementos fuera de los contenedores:
	- Empezar con un nuevo contenedor vacío.
	- Mientras haya elementos fuera que quepan en el contenedor actual:
		- Seleccionar un elemento con valor más grande que quepa en el contenedor y añadirlo.

Esta solución voraz posiblemente no produzca la solución óptima.

### Función `reparte`

In [2]:
def reparte(capacidad, elementos):
    """
    Reparte los elementos en varios contenedores de una misma capacidad.
    Los elementos son una colección de valores numéricos.
    El resultado es una lista de listas, cada lista representa los elementos de 
    un contenedor.
    """
    
    cerrados = []
    abiertos = []
    for elemento in elementos:
        destino = None
        for contenedor in abiertos:
            if sum(contenedor) + elemento <= capacidad:
                destino = contenedor
                break

        if destino is None:
            destino = []
            abiertos.append(destino)

        destino.append(elemento)

        if sum(destino) == capacidad:
            abiertos.remove(destino)
            cerrados.append(destino)

    return cerrados + abiertos


### Tests para el reparto de elementos

In [3]:
class TestReparte(unittest.TestCase):
    
    def estan_repartidos(self, capacidad, elementos, reparto):
        """
        Comprueba si los elementos están repartidos de acuerdo a una determinada 
        capacidad.
        """

        # En todos los contenedores se respeta la capacidad
        self.assertTrue(
            all(sum(contenedor) <= capacidad for contenedor in reparto))

        # Los elementos en el reparto coinciden con los originales
        self.assertEqual(
            Counter(elementos),
            Counter(elemento
                    for contenedor in reparto
                    for elemento in contenedor)
        )
        
    def test_simple(self):
        """ Primer caso de prueba simple """
        
        capacidad = 10
        elementos = 100*[1]
        reparto = reparte(capacidad, elementos)
        self.assertEqual(len(reparto), 10)
        self.estan_repartidos(capacidad, elementos, reparto)
        
    def tests_varios(self):
        """ Varios casos de prueba"""
        
        for capacidad, elementos, longitud in (
            (10, 100*[1], 10),
            (100, 1000*[1], 10),
            (10, 100*[9], 100),
            (10, 100*[4], 50),
            (10, 100*[3], 34),
            (10, 100*[6, 3], 100),
            (10, 100*[6, 5, 4], 150),
            (10, 100*[5, 1, 4, 3, 2], 150)
        ):
            reparto = reparte(capacidad, elementos)
            self.assertEqual(len(reparto), longitud)
            self.estan_repartidos(capacidad, elementos, reparto)

## Selección voraz de conjuntos

Se dispone de una secuencia de conjuntos. Los conjuntos pueden compartir elementos. La unión de todos los conjuntos en la secuencia es el conjunto _*S*_.

Se quiere una secuencia con algunos de esos conjuntos, tal que la unión de los elementos en esa secuencia también sea _*S*_. 

El objetivo es minimizar la longitud de la secuencia (el número de conjuntos seleccionados).

Se plantea la siguiente solución voraz:
- Se empieza con una secuencia vacía.
- Se selecciona el conjunto de la secuencia de entrada que tiene más elementos que no están en ninguno de los conjuntos ya seleccionados. 
- Se añade el conjunto seleccionado a la secuencia de salida.

La secuencia conseguida con esta solución voraz posiblemente no sea óptima.


### Función `seleccion_voraz`

In [4]:
def seleccion_voraz(conjuntos):
    """
    Dada una secuencia de conjuntos, devuelve otra, tal que los conjuntos 
    resultantes de la unión de los conjuntos de las secuencias de entrada y 
    salida coincidan.
    La selección de conjuntos es voraz: se selecciona el conjunto que tenga más 
    elementos que no estén en ninguno de los conjuntos seleccionados 
    previamente.
    Si varios conjuntos tienen el mismo número de elementos, se selecciona el
    que esté primero en la secuencia de entrada.
    La secuencia de salida será una lista, sus elementos deben aparecer en el 
    orden de selección.
    """
    
    abiertos = conjuntos.copy()
    cerrados = []
    resultado = []
    while True:
        maximo = [None, 0]
        for conjunto in abiertos:
            diferencia = len(conjunto.difference(cerrados))
            if diferencia > maximo[1]:
                maximo = [conjunto, diferencia]
        if maximo[0] is None:
            break
        else:
            abiertos.remove(maximo[0])
            cerrados.extend(maximo[0])
            resultado.append(maximo[0])
    return resultado


### Tests para la selección voraz

In [5]:
class TestSeleccionVoraz(unittest.TestCase):
    
    def test_1(self):
        self.assertEqual(
            seleccion_voraz([{1, 2, 3}, {2, 4}, {3, 4}, {4, 5}]), 
            [{1, 2, 3}, {4, 5}])       

    def test_2(self):
        self.assertEqual(
            seleccion_voraz([{1, 2, 3}, {2, 3, 4}, {3, 4, 5}]),
            [{1, 2, 3}, {3, 4, 5}]
        )
              
    def test_3(self):
        self.assertEqual(
            seleccion_voraz([{1, 2}, {2, 3}, {3, 4}, {4, 5}]),
            [{1, 2}, {3, 4}, {4, 5}]
        )

    def test_4(self):
        self.assertEqual(
            seleccion_voraz([{1, 2, 3, 4}, {5, 6, 7}, {8}, {9}, {1, 4, 7}, 
                             {8, 2, 5}, {9, 3, 6}]),
            [{1, 2, 3, 4}, {5, 6, 7}, {8}, {9}]
        )
        
    def test_5(self):
        self.assertEqual(
            seleccion_voraz([{1, 4, 7}, {1, 2, 3, 4}, {5, 6, 7}, {8}, {9},  
                             {8, 2, 5}, {9, 3, 6}]),
            [{1, 2, 3, 4}, {5, 6, 7}, {8}, {9}]
        )
        
    def es_seleccion(self, originales, seleccionados):
        """
        Comprueba si las uniones de originales y seleccionados contienen los 
        mismos elementos y todos los seleccionados están en los originales.
        """
        
        self.assertEqual(
            {elemento for conjunto in originales for elemento in conjunto},
            {elemento for conjunto in seleccionados for elemento in conjunto}
        )
        
        for conjunto in seleccionados:
            self.assertTrue(conjunto in originales)
        
    def test_multiplos(self):
        """Los conjuntos están formados por múltiplos de enteros"""
        
        for maximo, longitud in (
            (10, 4),
            (100, 25),
            (1000, 168)
        ):
            conjuntos = []
            for i in range(2, maximo):
                conjuntos.append(set(range(i, maximo, i)))
            seleccionados = seleccion_voraz(conjuntos)
            self.assertEqual(longitud, len(seleccionados))
            self.es_seleccion(conjuntos, seleccionados)

## Ejecución de tests

In [6]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

........
----------------------------------------------------------------------
Ran 8 tests in 9.971s

OK
